In [1]:
# =======================================================
# NOTE: Piping in Python (similar to %>% in R, | in Unix)
from datetime import datetime as dt
from itertools import chain
import math
import re

from bs4 import BeautifulSoup as bs
from matplotlib import test
import numpy as np
import pandas as pd
import requests
from sspipe import p, px

import tabloo                             # dataframe visualizer
# import dfply                            # like dplyr

'11.999999999999998'

In [ ]:
url = "https://slashdot.org/"

headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    # 'cache-control': 'max-age=0',
    "dnt": "1",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "cross-site",
    "sec-fetch-user": "?1",
    "sec-gpc": "1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.101 Safari/537.36",
}

r = requests.get(url, headers=headers)
soup = bs(r.text, "html.parser")

#### ┌─────────────────────────────────────────────────┐
#### │ Scraping one page                               │
#### └─────────────────────────────────────────────────┘

In [ ]:
# Title, link, rating
tit = [t.get_text() for t in soup.select("h2 span a")]

# Title of post
title = [j for i in soup.find_all("span", class_="story-title") for j in i.find("a")]

# Date/time of post
import dateutil.parser
# dateutil.parser.parse('string')
dated = [d.get_text() for d in soup.select("span.story-byline time")]
date = [re.sub("on|@", "", x).strip() for x in dated]
dt = [dt.strptime(d, "%A %B %d, %Y %I:%M%p") for d in date]

# External link to post
elink = [l.text.strip() for l in soup.select("h2 span span")]

# Comments on post
comments = (
    [x.get_text() for x in soup.select("span.comment-bubble a")]
    | p(np.array)
    | px.astype("int")
)

# Category of post
cat = [b.get("alt") for b in soup.find_all("img")] | p(list, p(filter, None, px))
# Using this sort of as a try except in case it were to not exist
category = [x.replace("Icon", "") for x in cat] | p(filter, None) | p(list)

# User who made the post
user = [
    u.get_text(" ", strip=True).replace("\n", "").replace("\t", "")
    for u in soup.select("span.story-byline")
]
user = [" ".join(a.split()) | p(re.findall, r"Postedby\s(\w+)", px) for a in user]

# Popularity of post (ratings? red?)
pop = [
    re.findall("'([a-zA-Z0-9,\s]*)'", prop["onclick"]) | px[1]
    for prop in soup.find_all("span", attrs={"alt": "Popularity"})
]

### ┌─────────────────────────────────────────────────┐
### │ Dataframe                                       │
### └─────────────────────────────────────────────────┘

In [ ]:
df = pd.DataFrame(
    {
        "title": title,
        "date": dt,
        "exlink": elink,
        "comments": comments,
        "category": category,
        "user": user,
        "popular": pop,
    }
)


tabloo.show(df)
# from operator import itemgetter
# df['user'] = df['user'] | p(map, p(itemgetter(0)), px) | p(list)

df["user"] = df["user"] | p(chain.from_iterable) | p(list)
df["exlink"] = df["exlink"] | px.str.replace(r"\(|\)", "", regex=True)

### ┌─────────────────────────────────────────────────┐
### │ Functions for scraping more than one page       │
### └─────────────────────────────────────────────────┘

In [11]:
import time

def get_page(url, sleep=False, prnt=False):
    response = requests.get(url)
    if not response.ok:
        print("Server Responded: ", response.status_code)
    else:
        soup = bs(response.text, "lxml")
        patt = re.compile(r'\d+')
        if sleep:
            time.sleep(3)
        if prnt:
            print(f'Number {patt.search(url).group(): ^5}done')
    return soup


def get_data(soup):
    pattern = re.compile(r"\([a-z0-9.\-]+[.](\w+)\)")

    try:
        title_ = [x.get_text(' ', strip=True) for x in soup.select("h2 span.story-title")]
        title = [pattern.sub('', x).strip() for x in title_]
    except:
        title = ""

    try:
        dated = [d.get_text(" ", strip=True) for d in soup.select("span.story-byline time")]
        date = [re.sub("on|@", "", x).strip() for x in dated]
    except:
        date = ""

    try:
        curls = {}
        ex = [x.get_text(' ', strip=True) for x in soup.select("h2 span.story-title")]

        for idx, u in enumerate(ex):
            if not pattern.search(u):
                curls[idx] = "Empty"
            else:
                curls[idx] = pattern.search(u).group()

        elink = list(curls.values())
    except:
        try:
            elink = [l.text.strip() for l in soup.select("h2 span span.no")]
        except:
            elink = ""

    try:
        comments = (
            [x.get_text() for x in soup.select("span.comment-bubble a")]
            | p(np.array)
            | px.astype("int"))
    except:
        comments = ""

    try:
        cat = ([b.get("alt") for b in soup.find_all("img")]
              | p(list, p(filter, None, px)))
        category = ([x.replace("Icon", "") for x in cat]
                    | p(filter, None)
                    | p(list))
    except:
        category = ""

    try:
        user = [u.get_text(" ", strip=True).replace("\n", "").replace("\t", "")
                for u in soup.select("span.story-byline")]
        user = [" ".join(a.split())
                | p(re.findall, r"Postedby\s(\w+)", px) for a in user]
    except:
        user = ""

    try:
        pop = [re.findall("'([a-zA-Z0-9,\s]*)'", prop["onclick"]) | px[1]
            for prop in soup.find_all("span", attrs={"alt": "Popularity"})]
    except:
        pop = ""

    temp =pd.DataFrame({
            "title": title,
            "date": date,
            "exlink": elink,
            "comments": comments,
            "category": category,
            "user": user,
            "popular": pop
        }
    )
    return temp

### ┌─────────────────────────────────────────────────┐
### │ Scraping more than one page                     │
### └─────────────────────────────────────────────────┘

In [79]:
from random import sample

base_url = "https://slashdot.org/?page="
urls = [base_url + str(i) for i in range(1, 100)]

test_u = urls | p(sample, 2)


data = [get_data(get_page(x, sleep=True, prnt=True)) for x in urls] | p(pd.concat, px)

data = data.reset_index(drop=True)

data.to_csv('df.csv', index=True, columns=data.columns.values)

### ┌─────────────────────────────────────────────────┐
### │ Reading back in & cleaning                      │
### └─────────────────────────────────────────────────┘

In [ ]:
df = pd.read_csv('df.csv', index_col=0)

df.columns.values
df.dtypes

# Date
df['date'] = [' '.join(x.split(' ')[1:]) | p(dt.strptime, "%B %d, %Y %I:%M%p") for x in df['date']]
df["exlink"] = df["exlink"] | px.str.replace(r"\(|\)", "", regex=True)
df["category"] = df["category"].astype("category")
df["user"] = df["user"] | px.str.replace(r"\[|\]|\'", "", regex=True)
df["popular"] = df["popular"].astype("category")

df.dtypes

### ┌─────────────────────────────────────────────────┐
### │ Scraping post pages for comments                │
### └─────────────────────────────────────────────────┘

In [ ]:
# Main page to post page
test = urls[0]
soup = get_page(test)

In [ ]:
# Internal links to other posts
ilink_ = [x['href'] for x in soup.select('h2 span a')] | p(filter, None) | p(list)
ilink = [re.sub("//", "", x) for x in ilink_[::2] if x.startswith("//")]

In [ ]:
# Post page
ppage = "https://news.slashdot.org/story/21/02/11/0029259/wall-street-fund-wants-to-hire-rwallstreetbets-users-to-help-pick-meme-stocks#comments"

In [ ]:
psoup = get_page(ppage)

In [ ]:
# Title / External Link / Comments
tec = [x.get_text() for x in psoup.select("h2 span a")]

In [ ]:
ptitle, pelink = [x.get_text(' ', strip=True) for x in psoup.select("h2 span.story-title a")]
pcomm = [x.get_text() for x in psoup.select(("span.comment-bubble"))] | px[0]